In [1]:
import pathlib
import os
if not "changed_working_directory" in locals():
    changed_working_directory = True
    os.chdir(pathlib.Path().resolve().parent)
    
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import os
import pickle
import time
import tensorflow as tf
import shutil
import numpy as np
from math import ceil
from src import constants as const
import utils.timer
import helpers

dataset_train_path = os.path.join(const.dir.data(), "df_adult_edited_train.csv")
dataset_test_path = os.path.join(const.dir.data(), "df_adult_edited_test.csv")

data_train = pd.read_csv(dataset_train_path)
data_test = pd.read_csv(dataset_test_path)
columns_num, = np.where(data_train.dtypes != "object")

timer_path = os.path.join(const.dir.timers(), "timer_tgan.pkl")
retrain = True

if not os.path.exists(timer_path):
    timer = utils.timer.Timer()
    timer.save(timer_path)
    del timer
    
from tgan.model import TGANModel

dir_data_gen_tgan = os.path.join(const.dir.data_gen(), "TGAN")
model_dir = os.path.join(const.dir.models(), "tgan")

batch_size = 500
retrain=True

In [3]:
import tensorflow as tf
print(tf.__version__)
tf.test.is_gpu_available()

1.15.5


True

In [12]:
?TGANModel

In [18]:
if retrain:
    tf.reset_default_graph()
    if os.path.exists(model_dir):
        shutil.rmtree(model_dir)
    timer_tgan = utils.timer.load(timer_path)
    timer_tgan.start("TGAN")
    tgan = TGANModel(columns_num, output = model_dir,
                     gpu="0",
                     max_epoch = 40, steps_per_epoch = data_train.shape[0] / batch_size, batch_size=batch_size,
                     num_dis_hidden = 256, num_dis_layers=2, z_dim=128,
                    restore_session = False, save_checkpoints = True)
    tgan.fit(data_train)
    timer_tgan.stop("TGAN")
    timer_tgan.save(timer_path)

PermissionError: [WinError 5] Ingen tilgang: 'S:/arneir/Master-thesis-storage/models\\tgan'

In [14]:
tgan.save(os.path.join(model_dir, "model.pkl"))

[0315 10:50:12 @model.py:813] Model saved successfully.


In [6]:
tgan = TGANModel.load(os.path.join(model_dir, "model.pkl"))












Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.

Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
[0315 10:57:59 @registry.py:126] gen/LSTM/00/FC input: [500, 100]

Instructions for updating:
Please use `layer.__call__` method instead.
[0315 10:57:59 @registry.py:134] gen/LSTM/00/FC output: [500, 100]
[0315 10:57:59 @registry.py:126] gen/LSTM/00/FC2 input: [500, 100]
[0315 10:57:59 @registry.py:134] gen/LSTM/00/FC2 output: [500, 1]
[0315 10:57:59 @registry.py:126] gen/LSTM/01/FC input: [500, 100]
[0315 10:57:59 @registry.py:134] gen/LSTM/01/FC output: [500, 100]
[0315 10:57:59 @registry.py:126] gen/LSTM/01/FC2 input: [500, 100]
[0315 10:57:59 @registry.py:134] gen/LSTM/01/FC2 output: [500, 5]
[0315 10:57:59 @registry.py:126] gen/LSTM/01/FC3 input: [500, 5]

[0315 10:58:00 @registry.py:126] gen/LSTM/10/FC2 input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/10/FC2 output: [500, 5]
[0315 10:58:00 @registry.py:126] gen/LSTM/10/FC3 input: [500, 5]
[0315 10:58:00 @registry.py:134] gen/LSTM/10/FC3 output: [500, 100]
[0315 10:58:00 @registry.py:126] gen/LSTM/11/FC input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/11/FC output: [500, 100]
[0315 10:58:00 @registry.py:126] gen/LSTM/11/FC2 input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/11/FC2 output: [500, 2]
[0315 10:58:00 @registry.py:126] gen/LSTM/11/FC3 input: [500, 2]
[0315 10:58:00 @registry.py:134] gen/LSTM/11/FC3 output: [500, 100]
[0315 10:58:00 @registry.py:126] gen/LSTM/12/FC input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/12/FC output: [500, 100]
[0315 10:58:00 @registry.py:126] gen/LSTM/12/FC2 input: [500, 100]
[0315 10:58:00 @registry.py:134] gen/LSTM/12/FC2 output: [500, 1]
[0315 10:58:00 @registry.py:126] gen/LSTM/13/FC input: [500, 100]
[03

In [15]:
tgan_synthetic_data_train = tgan.sample(ceil(data_train.shape[0] / batch_size) * batch_size).head(data_train.shape[0])
tgan_synthetic_data_train.to_csv(os.path.join(dir_data_gen_tgan, "train_dataset.csv"), index=False)
tgan_synthetic_data_test = tgan.sample(ceil(data_test.shape[0] / batch_size) * batch_size).head(data_test.shape[0])
tgan_synthetic_data_test.to_csv(os.path.join(dir_data_gen_tgan, "test_dataset.csv"), index=False)

  6%|#####                                                                               |30/500[00:00<00:11,39.83it/s]


In [8]:
print(tgan_synthetic_data_train.shape[0], "vs", data_train.shape[0])
print(tgan_synthetic_data_test.shape[0], "vs", data_test.shape[0])

30162 vs 30162
15060 vs 15060


In [16]:
tgan_synthetic_data_train

,age,workclass,fnlwgt,education,educational.num,marital.status,occupation,race,gender,capital.gain,capital.loss,hours.per.week,native.country,income
0,56.420453,Self-emp-not-inc,14442.746892,HS-grad,9.062582,Married,Exec-managerial,White,Male,-0.002000,0.002000,53.888244,US,>50K
1,25.365530,Private,13045.426430,HS-grad,8.543215,Never-married,Adm-clerical,White,Male,-0.001999,0.001995,40.001716,US,<=50K
2,35.669026,Self-emp-not-inc,131569.054474,Some-college,9.569303,Divorced,Craft-repair,White,Male,-0.002000,0.001949,75.647998,US,>50K
3,72.819391,Private,60094.733618,Assoc-voc,3.226417,Never-married,Exec-managerial,White,Female,-0.002000,-0.001897,30.357612,Non-US,<=50K
4,36.968999,Private,22194.396616,HS-grad,8.642888,Divorced,Handlers-cleaners,White,Male,-0.002000,0.001975,40.000643,US,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30157,46.078009,Private,120776.562197,Bachelors,12.120272,Married,Machine-op-inspct,White,Male,274.274323,0.001921,39.999426,US,<=50K
30158,36.191944,Government,159833.161333,HS-grad,8.780404,Married,Machine-op-inspct,White,Male,-0.002000,0.001984,50.791902,US,>50K
30159,43.094701,Government,146723.984332,Some-college,9.684307,Divorced,Protective-serv,White,Female,-0.002000,0.001989,40.000888,US,<=50K
30160,58.642034,Private,141208.238251,Bachelors,12.964556,Married,Adm-clerical,White,Male,-0.002000,0.001974,80.767438,US,<=50K


In [20]:
import helpers
helpers.eval.fit_and_evaluate_xgboost(tgan_synthetic_data_train, data_test, tree_method="hist")

NameError: name 'tgan_synthetic_data_train' is not defined